In [1]:
import pandas as pd

Com o arquivo `dicionario_tratado.csv` em mãos (gerado no etl_dicionario), vamos prosseguir com a leitura do .txt.
Para iniciar, o pandas fornece uma função especifica para estes modelos, e vamos usar o nosso csv para tal.

In [2]:
dic = pd.read_csv("dicionario_tratado.csv")
print(dic.head())

  Posição inicial  Tamanho Código da variável Quesito     Descrição quesito  \
0               1      2.0              V0001     NaN  Unidade da Federação   
1               1      2.0              V0001     NaN  Unidade da Federação   
2               1      2.0              V0001     NaN  Unidade da Federação   
3               1      2.0              V0001     NaN  Unidade da Federação   
4               1      2.0              V0001     NaN  Unidade da Federação   

  Tipo categoria Descrição categoria  
0             11            Rondônia  
1             12                Acre  
2             13            Amazonas  
3             14             Roraima  
4             15                Pará  


Vamos começar lidando com as linhas com mesmo código da variável, mas com valores de variáveis diferentes

In [3]:
dic_vars = (
    dic.groupby('Código da variável', as_index=False)
       .agg({'Posição inicial':'first', 'Tamanho':'first'})
       .rename(columns={'Código da variável':'var'})
)
# garantir inteiros
dic_vars['Posição inicial'] = dic_vars['Posição inicial'].astype('Int64')
dic_vars['Tamanho'] = dic_vars['Tamanho'].astype('Int64')

# Posições
dic_vars['start0'] = dic_vars['Posição inicial'] - 1
dic_vars['end0']   = dic_vars['start0'] + dic_vars['Tamanho']

colspecs = list(zip(dic_vars['start0'].astype(int), dic_vars['end0'].astype(int)))
names = dic_vars['var'].tolist()

print(names[0])
print(colspecs[0])


A001
(34, 35)


Agora, vamos criar o mapping para as variáveis

In [4]:
dic['Tipo categoria'] = dic['Tipo categoria'].astype('string')
dic['Descrição categoria'] = dic['Descrição categoria'].astype('string')

code_maps = {}
for var, g in dic.groupby('Código da variável'):
    # só linhas que realmente têm categoria
    g = g.dropna(subset=['Tipo categoria', 'Descrição categoria'])
    if not g.empty:
        code_maps[var] = dict(zip(g['Tipo categoria'], g['Descrição categoria']))

print(code_maps['V0001'])


{'11': 'Rondônia', '12': 'Acre', '13': 'Amazonas', '14': 'Roraima', '15': 'Pará', '16': 'Amapá', '17': 'Tocantins', '21': 'Maranhão', '22': 'Piauí', '23': 'Ceará', '24': 'Rio Grande do Norte', '25': 'Paraíba', '26': 'Pernambuco', '27': 'Alagoas', '28': 'Sergipe', '29': 'Bahia', '31': 'Minas Gerais', '32': 'Espírito Santo', '33': 'Rio de Janeiro', '35': 'São Paulo', '41': 'Paraná', '42': 'Santa Catarina', '43': 'Rio Grande do Sul', '50': 'Mato Grosso do Sul', '51': 'Mato Grosso', '52': 'Goiás', '53': 'Distrito Federal'}


Finalmente, podemos seguir com a leitura dos microdados. Não vou fazer o parse de todas as variáveis com a função acima porque adicionaria muito tempo, utilizaremos a função apenas para auxiliar no entendimento.

In [5]:
df = pd.read_fwf(
    'PNS_2019.txt',
    colspecs=colspecs,
    names=names,
    dtype=str,
    encoding='utf-8',
    nrows=1000
)

print(df.head())

  A001 A002010 A003010 A004010 A005010 A005012 A00601 A009010 A01001 A011  \
0    1       1       1       1       1     NaN      1       1     05   02   
1    1       1       1       1       1     NaN      1       1     05   02   
2    1       1       1       1       1     NaN      1       1     05   02   
3    1       1       1       1       1     NaN      1       1     05   02   
4    1       1       1       1       1     NaN      1       1     05   02   

   ... Z007 Z008 Z009 Z010 Z011 Z012 Z013 Z01401 Z01402 k055  
0  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN  NaN  
1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN  NaN  
2  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN  NaN  
3  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN  NaN  
4  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN    NaN    NaN  NaN  

[5 rows x 1088 columns]


Vamos começar filtrando os idosos. Olhando no dicionário, o código para a idade é `C008`.

In [6]:
print(code_maps["C008"])
print(df["C008"].head(10))

{'000 a 130': 'Idade (em anos)'}
0    055
1    069
2    031
3    009
4    006
5    004
6    033
7    017
8    015
9    019
Name: C008, dtype: object


In [8]:
df["C008"] = pd.to_numeric(df["C008"], errors="coerce") # garantindo que valores são numeros e não strings
idosos = df[df["C008"] >= 60] # filtrando os maiores que 60
print(idosos.shape)

(112, 1088)


Como podem ver, ao filtrar os idosos, passamos de 1000 entries para 118. A partir destes idosos, podemos seguir fazendo outras análises e filtros.